<a href="https://colab.research.google.com/github/Rafael-Maranhao/portfolio/blob/main/notebook_startup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Levantamento de dados: Filial de startup

Levantamento de dados para uma startup que atua no setor agrícola.
Eles desejavam abrir um novo escritório, que servirá para tratar tanto de assuntos comerciais como para recrutar novos profissionais.

Neste sentido o levantamento trouxe as mais fortes concorrentes entre as cidades para hospedar uma filial da empresa:

1. Deveria está localizado em uma cidade que tem área produtiva de soja;
2. Deve está em uma das 100 maiores cidades do Brasil (em número de habitantes);

*   Item da lista
*   Item da lista



In [ ]:
import urllib3
import json
import pandas as pd
import os

# Configuração do pandas para mostrar todas as colunas e linhas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Função para obter dados da API com tratamento de erro
def get_data(url, nome_arquivo):
    http = urllib3.PoolManager()
    try:
        r = http.request('GET', url)
        dados = json.loads(r.data)

        # Salva os dados brutos em JSON
        with open(f'{nome_arquivo}.json', 'w') as f:
            json.dump(dados, f)

        # Converte para CSV se for estrutura tabular
        try:
            df = pd.json_normalize(dados)
            df.to_csv(f'{nome_arquivo}.csv', index=False)
            print(f"Dados salvos em {nome_arquivo}.csv")
        except:
            print(f"Dados salvos apenas em {nome_arquivo}.json (estrutura não tabular)")

        return dados
    except Exception as e:
        print(f"Erro ao acessar API: {str(e)}")
        return None

# Obter dados e salvar arquivos brutos
print("Coletando dados de população...")
dados_populacao = get_data(
    'https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2021/variaveis/9324?localidades=N6[all]',
    'dados_populacao_brutos'
)

print("\nColetando dados de área de soja...")
dados_soja = get_data(
    'https://servicodados.ibge.gov.br/api/v3/agregados/1612/periodos/2021/variaveis/109?localidades=N6[all]&classificacao=81[2713]',
    'dados_soja_brutos'
)

# [O RESTANTE DO SEU SCRIPT PERMANECE EXATAMENTE IGUAL...]
# Função para processar dados de população
def processar_populacao(dados):
    municipios = []
    if dados and len(dados) > 0:
        for item in dados[0]['resultados'][0]['series']:
            try:
                municipios.append({
                    'codigo': item['localidade']['id'],
                    'municipio': item['localidade']['nome'],
                    'populacao': int(item['serie']['2021'])
                })
            except:
                continue
    return pd.DataFrame(municipios)

# Função para processar dados de soja
def processar_soja(dados):
    municipios = []
    if dados and len(dados) > 0:
        for item in dados[0]['resultados'][0]['series']:
            try:
                valor = item['serie']['2021']
                # Considera apenas valores numéricos positivos
                if valor not in ['-', '...', '0']:
                    area = float(valor)
                    if area > 0:
                        municipios.append({
                            'codigo': item['localidade']['id'],
                            'municipio': item['localidade']['nome'],
                            'area_soja': area
                        })
            except:
                continue
    return pd.DataFrame(municipios)

# Processar os dados
print("Processando dados...")
df_pop = processar_populacao(dados_populacao)
df_soja = processar_soja(dados_soja)

# Filtrar os 100 mais populosos
top_100_pop = df_pop.nlargest(100, 'populacao')

# Juntar com dados de soja
df_final = pd.merge(top_100_pop, df_soja, on=['codigo', 'municipio'], how='inner')

# Ordenar por população (decrescente)
df_final = df_final.sort_values('populacao', ascending=False)

# Formatar a saída
df_final['area_soja'] = df_final['area_soja'].apply(lambda x: f"{x:,.0f} ha")
df_final['populacao'] = df_final['populacao'].apply(lambda x: f"{x:,}")

# Resultado final
print("\n" + "="*80)
print("MUNICÍPIOS RECOMENDADOS PARA O NOVO ESCRITÓRIO".center(80))
print("(Entre os 100 mais populosos com produção de soja > 0)".center(80))
print("="*80 + "\n")

# Configurar a formatação da tabela
from tabulate import tabulate

# Formatar os dados para exibição
df_display = df_final[['municipio', 'populacao', 'area_soja']].copy()
df_display.columns = ['MUNICÍPIO', 'POPULAÇÃO', 'ÁREA DE SOJA (ha)']  # Renomear colunas

# Usar tabulate para melhor formatação
print(tabulate(df_display,
               headers='keys',
               tablefmt='pretty',
               stralign='left',
               showindex=False,
               colalign=('left', 'right', 'right')))

print("\n" + "="*80)
print(f"Total de municípios recomendados: {len(df_final)}".center(80))
print("="*80)

# Salvar em CSV (mantendo o original)
df_final.to_csv('municipios_recomendados.csv', index=False)
print("\nArquivo 'municipios_recomendados.csv' salvo com todos os dados.")

Coletando dados de população...
Dados salvos em dados_populacao_brutos.csv

Coletando dados de área de soja...
Dados salvos em dados_soja_brutos.csv
Processando dados...

                 MUNICÍPIOS RECOMENDADOS PARA O NOVO ESCRITÓRIO                 
             (Entre os 100 mais populosos com produção de soja > 0)             

+----------------------------+-----------+-------------------+
| MUNICÍPIO                  | POPULAÇÃO | ÁREA DE SOJA (ha) |
+----------------------------+-----------+-------------------+
| Brasília - DF              | 3,094,325 |         80,000 ha |
| Goiânia - GO               | 1,555,626 |          1,150 ha |
| Porto Alegre - RS          | 1,492,530 |            398 ha |
| Campinas - SP              | 1,223,237 |            800 ha |
| Campo Grande - MS          |   916,001 |         94,000 ha |
| Ribeirão Preto - SP        |   720,116 |            700 ha |
| Uberlândia - MG            |   706,597 |         58,000 ha |
| Sorocaba - SP              |   695